In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.special import p_roots

# Forward modelling
Use Gauss-Legendre quadrature with 40 points to model integration of a piecewise-linear function $f(t_k)$.
$$
x_k = f(t_k)
$$
$$
y_j = \mathcal{L}f(s_j)
$$
$$
a_{jk} = w_k e^{-s_j t_k}
$$

The points $s_j$ are logarithmically distributed:
$$
s_j = 10 ^ {-1 + \frac{j - 1}{20}}
$$
$$
1 \leq j \leq 40
$$
The Laplace transform of the given function is given by:
$$
\mathcal{L}f(s_j) = \frac{1}{2 s_j^2} (2 - 3 e^{-s_j} + e^{-3 s_j})
$$

In [2]:
n = 40

In [3]:
print('Evaluating n = %d points to be used by Gauss-Legendre quadrature rule.' % n)
[T0, W0] = p_roots(n)
T = [2.5 * (1 + t0) for t0 in T0] # Transforming integration limits from [-1, 1] to [0, 5]
W = 0.5 * (5 - 0) * W0

# Compute the loading vector x = f(tk)
x = np.zeros((n, 1))
for k, tk in enumerate(T):
    if (tk >= 0) & (tk < 1):
        x[k] = tk
    elif tk < 3:
        x[k] = 1.5 - 0.5 * tk
    else:
        x[k] = 0

print('Evaluating logarithmically distributed points to measure y on.')
S = [10 ** (-1 + (j - 1) / 20) for j in range(1, n + 1)]

# Compute the analytical (true) value of the Laplace transform y = Lf(sj)
y_list = [1 / (2 * sj ** 2) * (2 - 3 * math.exp(-sj) + math.exp(-3 * sj)) for sj in S]
y = np.array(y_list).reshape((n, 1))

# Generate the forward model matrix A
A = np.zeros((n, n))
for j in range(n):
    for k in range(n):
        A[j][k] = W[k] * math.exp(-S[j] * T[k])
print('The determinant of the matrix A is %.2f.' % np.linalg.det(A))

# Check if the forward model is correct
e = A @ x - y
print('Min. norm error of true solution = %.2f.' % np.linalg.norm(e))

Evaluating n = 40 points to be used by Gauss-Legendre quadrature rule.
Evaluating logarithmically distributed points to measure y on.
The determinant of the matrix A is 0.00.
Min. norm error of true solution = 0.00.


# Inverse modelling
Case-1: Direct solution of Ax = y

In [4]:
Ainv = np.linalg.inv(A)
xmin = Ainv @ y

In [5]:
fig1 = plt.figure(figsize=(9, 12), dpi=65)
fig1.suptitle('Figure 2.1', fontsize=16)
fig1.add_subplot(311)
plt.plot(T, x, '-o')
plt.xlabel('$t_k$', fontsize=14)
plt.ylabel('$f(t_k)$', fontsize=14)
plt.xlim([0, 5])
fig1.add_subplot(312)
plt.plot(S, y, '-o')
plt.xlabel('$s_j$', fontsize=14)
plt.ylabel('$\mathcal{L}f(s_j)$', fontsize=14)
fig1.add_subplot(313)
plt.plot(T, xmin, '-^')
plt.xlabel('$t_k$', fontsize=14)
plt.ylabel('$f_{Direct}(t_k)$', fontsize=14)
plt.xlim([0, 5])
plt.show()

Case-2A: Solving Ax = y using Truncated SVD

In [6]:
U, L, VT = np.linalg.svd(A, full_matrices=0)

P = [19, 20, 21]
Xp = []
for p in P:
    x_p = VT[:p, :].T @ np.linalg.inv(np.diag(L[:p])) @ U[:, :p].T @ y
    Xp.append(x_p)

Case-2B: Solving Ax = y + $\eta$ using truncated SVD

In [7]:
# Generate noisy measurement data
y_inf = np.max(np.abs(y))
sigma = 0.01 * y_inf
eta = np.random.normal(scale=sigma, size=n).reshape(-1, 1)
y_noisy = y + eta

In [8]:
# Apply TSVD on noisy measurement data
Pn = [4, 5, 6]
Xnp = []
for p in Pn:
    xnp = VT[:p, :].T @ np.linalg.inv(np.diag(L[:p])) @ U[:, :p].T @ y_noisy
    Xnp.append(xnp)

In [9]:
plt.figure()
plt.plot(range(1, n+1), np.log(L), '*')
plt.axhline(y=np.log(1e-16), ls='-', color='k')
plt.axhline(y=np.log(sigma), ls='--', color='k')
plt.xlabel('i', fontsize=14)
plt.ylabel('$\sigma_i$', fontsize=14)
plt.grid()
plt.title('Figure 2.2', fontsize=16)
plt.show()

fig3 = plt.figure(figsize=(8, 6), dpi=120)
fig3.suptitle('Figure 2.3', fontsize=16)
fig3.add_subplot(211)
style = ['.-', '--', '-']
for i, xp in enumerate(Xp):
    plt.plot(T, xp, style[i], label=P[i])
plt.ylabel('$f(t_k)$', fontsize=14)
plt.xlim([0, 5])
plt.legend()
fig3.add_subplot(212)
style = ['.-', '--', '-']
for i, xnp in enumerate(Xnp):
    plt.plot(T, xnp, style[i], label=Pn[i])
plt.xlabel('$t_k$', fontsize=14)
plt.ylabel('$f(t_k)$', fontsize=14)
plt.xlim([0, 5])
plt.legend()
plt.show()

Case-3: Solving Ax = y + $\eta$ using Conjugate Gradient

In [10]:
def conjugateGradientDiscrepancy(A, b, x, noise_estimate, i_max=1e3):
    i = 0
    r = b - A.dot(x)
    d = r
    delta = np.linalg.norm(r); ResNorm = delta ** 2
    X = []; X.append(x)
    while (i < i_max) & (delta > noise_estimate):
        q = A.dot(d)
        alpha = delta ** 2 / (d.T.dot(q))
        x = x + alpha * d
        r = r - alpha * q
        ResNormOld = ResNorm
        delta = np.linalg.norm(r); ResNorm = delta ** 2
        beta = ResNorm / ResNormOld
        d = r + beta * d
        X.append(x)
        i += 1
    return x, np.array(X), i

In [12]:
x_0 = np.zeros((n, 1))
noise_estimate = math.sqrt(n) * sigma

_, XCG, i_max = conjugateGradientDiscrepancy(A, y, x_0, noise_estimate=0, i_max=9)
print('Total no. of iterations tested =', i_max)

xCG, _, n_steps = conjugateGradientDiscrepancy(A, y, x_0, noise_estimate=noise_estimate, i_max=9)
print('No. of iterations to match discrepancy =', n_steps)

Total no. of iterations tested = 9
No. of iterations to match discrepancy = 7


In [19]:
np.shape(XCG[1:,:,:])

(9, 40, 1)

In [28]:
fig4 = plt.figure(figsize=(9, 9), dpi=100)
fig4.suptitle('Figure 2.14', fontsize=16)
for k, xCG_k in enumerate(XCG[1:, :, :], start=1):
    fig4.add_subplot(3, 3, k)
    plt.plot(T, x, lw=0.5)
    plt.plot(T, xCG_k, lw=2)
    plt.title('Iteration = %d' % k)
    plt.xlim([0, 5])
    plt.ylim([-0.5, 1.5])
fig4.tight_layout()
plt.show()